<a href="https://colab.research.google.com/github/jhonnycodedev/data-science/blob/jhonnycodedev/Lista%20NPS%20Positivo_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [3]:
from google.colab import files
uploaded = files.upload()


Saving Lista NPS Positivo_V4.xlsx to Lista NPS Positivo_V4.xlsx


In [4]:
import io

# Substitua 'nome_do_arquivo.xlsx' pelo nome do seu arquivo
df = pd.read_excel(io.BytesIO(uploaded['Lista NPS Positivo_V4.xlsx']), engine='openpyxl')


In [5]:
# Verificando as primeiras linhas
#print(df.head())

# Resumo estatístico
#print(df.describe())

# Informações sobre o DataFrame
#print(df.info())


In [22]:
# Função para classificar as notas
df['target'] = df['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))


In [24]:
df.groupby('target').count()

In [31]:
#import pandas as pd

# Carregar o arquivo Excel
#file_path = 'Lista NPS Positivo_V4.xlsx'
#df = pd.read_excel(file_path, sheet_name='answers')

# Classificar as notas
if 'nota' in df.columns:
    df['target'] = df['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))

# Filtrar para "Grupo de Produto" == "Grupo 2"
grupo_2_df = df[df['Grupo de Produto'] == 'Grupo 2']

# Definir as regiões com base nos estados
regioes = {
    "Norte": ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    "Nordeste": ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    "Centro-Oeste": ['DF', 'GO', 'MT', 'MS'],
    "Sul": ['PR', 'RS', 'SC'],
    "Sudeste": ['ES', 'MG', 'RJ', 'SP']
}

# Função para definir a região com base no estado
def obter_regiao(estado):
    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao
    return 'Outros'  # Para estados fora das regiões definidas

# Aplicar a função para criar a coluna 'regiao'
grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)

# Extrair o ano da data de resposta e manter a coluna 'Periodo de Pesquisa'
grupo_2_df['ano_resposta'] = pd.to_datetime(grupo_2_df['data_resposta']).dt.year

# Agrupar por 'regiao', 'ano_resposta', 'Periodo de Pesquisa' e 'target'
# Para contar as ocorrências de cada avaliação (Promotor, Neutro, Detrator) por ano e período
resultado_por_ano_periodo = grupo_2_df.groupby(['regiao', 'ano_resposta', 'Periodo de Pesquisa', 'target']).size().unstack(fill_value=0)

# Exibir o resumo resultante
print(resultado_por_ano_periodo)

# (Opcional) Salvar o resultado em um novo arquivo
#resultado_por_ano_periodo.to_excel('Resumo_Impacto_Notas_Grupo_2_Por_Ano_e_Periodo.xlsx')


target                                         Detrator  Neutro  Promotor
regiao       ano_resposta Periodo de Pesquisa                            
Centro-Oeste 2022          3 a 6 M                    0       1         0
                          12 a 18 M                   4       3        15
                          18 a 30 M                   0       6        16
                          3 a 6 M                     3       6        16
                          6 a 12 M                    2       6        19
...                                                 ...     ...       ...
Sul          2023         6 a 12 M                    5      16        44
             2024         12 a 18 M                   4       9        39
                          18 a 30 M                   9       7        22
                          3 a 6 M                     4      19        82
                          6 a 12 M                    8      24        77

[79 rows x 3 columns]


<ipython-input-31-c292ee8caf57>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)
<ipython-input-31-c292ee8caf57>:34: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  grupo_2_df['ano_resposta'] = pd.to_datetime(grupo_2_df['data_resposta']).dt.year
<ipython-input-31-c292ee8caf57>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [30]:
#import pandas as pd

# Carregar o arquivo Excel
#file_path = 'Lista NPS Positivo_V4.xlsx'
#df = pd.read_excel(file_path, sheet_name='answers')

# Classificar as notas
if 'nota' in df.columns:
    df['target'] = df['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))

# Filtrar para o "Grupo de Produto" == "Grupo 2"
grupo_2_df = df[df['Grupo de Produto'] == 'Grupo 2']

# Definir as regiões com base nos estados
regioes = {
    "Norte": ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    "Nordeste": ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    "Centro-Oeste": ['DF', 'GO', 'MT', 'MS'],
    "Sul": ['PR', 'RS', 'SC'],
    "Sudeste": ['ES', 'MG', 'RJ', 'SP']
}

# Criar uma coluna para a região
def obter_regiao(estado):
    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao
    return 'Outros'  # Caso o estado não pertença a nenhuma das regiões

grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)

# Extrair o ano de 'data_resposta' para análise temporal
grupo_2_df['ano_resposta'] = pd.to_datetime(grupo_2_df['data_resposta']).dt.year

# Criar um resumo das notas por região e ano
resultado_por_regiao_ano = grupo_2_df.groupby(['regiao', 'ano_resposta', 'target']).size().unstack(fill_value=0)

# Exibir o resumo
print(resultado_por_regiao_ano)

# (Opcional) Salvar o resultado em um novo arquivo para análise detalhada
#resultado_por_regiao_ano.to_excel('Resumo_Impacto_Notas_Grupo_2.xlsx')



target                     Detrator  Neutro  Promotor
regiao       ano_resposta                            
Centro-Oeste 2022                 9      22        66
             2023                21      26       155
             2024                18      14        80
Nordeste     2022                 4       8        26
             2023                 3      17        71
             2024                 4      13        93
Norte        2022                 2       5        18
             2023                 7      16        58
             2024                 7       7        48
Outros       2022                23      29        89
             2023                10      21        60
             2024                25      49       171
Sudeste      2022                21      43        83
             2023                32      66       195
             2024                18      38       191
Sul          2022                32      61       124
             2023           

<ipython-input-30-8ef460c0a1e9>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)
<ipython-input-30-8ef460c0a1e9>:33: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  grupo_2_df['ano_resposta'] = pd.to_datetime(grupo_2_df['data_resposta']).dt.year
<ipython-input-30-8ef460c0a1e9>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co